# Forward dPL Multi-layer Terzaghi's 1D Consolidation Model

-- Land subsidence project: 15 March 2025 --

---


#### 1. Data Preperations

Essentially, the internals of what will eventually be the data loader for this model.

In [9]:
### Load in data
import sys
import os
import numpy as np
import torch
import pickle

sys.path.append('../../')
sys.path.append('../../dMG')  # Add the dMG root directory.
sys.path.append(os.path.abspath('..'))  # Add the parent directory of `scripts` to the path.

from scripts import load_config


#------------------------------------------#
# Define model settings here.
CONFIG_PATH = '/projects/mhpi/leoglonz/dPLT/src/dMG/conf/config_ls.yaml'
TEST_SPLIT = 0.2
#------------------------------------------#



config = load_config(CONFIG_PATH)


# Load data
with open(config['observations']['train_path'], 'rb') as f:
    data_dict = pickle.load(f)


# Normalize attributes for NN:
attrs = data_dict['attributes']
for i in range(attrs.shape[-1]):
    attrs[:, :, i] = (attrs[:, :, i] - attrs[:, :, i].mean()) \
        / attrs[:, :, i].std()
    
data_dict['xc_nn_norm'] = attrs


# Train-test split + convert to torch tensors
train_dataset = {}
test_dataset = {}
split = int(len(data_dict['forcing']) * (1 - TEST_SPLIT))

for key in data_dict.keys():
    train_dataset[key] = torch.tensor(
        data_dict[key][:split,],
        dtype=config['dtype'],
        device=config['device'],
    )
    test_dataset[key] = torch.tensor(
        data_dict[key][split:,],
        dtype=config['dtype'],
        device=config['device'],
    )

# Reshape to 3d
shape = train_dataset['xc_nn_norm'].shape
train_dataset['xc_nn_norm'] = train_dataset['xc_nn_norm'].reshape(
    shape[0], 
    shape[1],
    shape[2] * shape[3],
)

shape = test_dataset['xc_nn_norm'].shape
test_dataset['xc_nn_norm'] = test_dataset['xc_nn_norm'].reshape(
    shape[0], 
    shape[1],
    shape[2] * shape[3],
)   

#### 2. Load dPL Scheme 

Physical model (Terzaghi's equation) + LSTM


In [14]:
import importlib

from dMG import load_nn_model
from src.dMG.models.phy_models.terzaghi import TerzaghiMultiLayer as dPLT
importlib.reload(sys.modules['src.dMG.models.phy_models.terzaghi'])
importlib.reload(sys.modules['dMG'])


model = dPLT(config['dpl_model']['phy_model'], device=config['device'])
nn = load_nn_model(
    model,
    config['dpl_model'],
    device=config['device'],
)

#### 3. Forward Model


In [16]:
parameters = nn(train_dataset['xc_nn_norm'])

# print(nn)
# print(f"Input shape {train_dataset['xc_nn_norm'].shape}")
# print(f"Output shape {parameters.shape}")

predictions = model(
    train_dataset,
    parameters,
)

/projects/mhpi/leoglonz/dPLT/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(


AttributeError: 'TerzaghiMultiLayer' object has no attribute 'dfLayers'

In [21]:
parameters.shape

torch.Size([109, 16, 8])